In [1]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import collect_set, collect_list, lit, sum, udf, concat_ws, col, count, abs


In [86]:
#df_ground_truth = spark.read.parquet('../output/ifood/dataset_5/info_session/ground_truth/').sort('click_timestamp', 'account_id')
df = spark.read.parquet('../output/ifood/dataset_3/session_train_buys_visits_0.10_k=0_s=1000')
df = df.select("account_id", "buy", "user_total_buys", "user_total_visits", 
               "item_total_buys", "item_total_visits", "hist_buys", "hist_visits")
key = "4b042feb-95d3-4710-b677-24d366895c44"
df = df.filter(df.account_id == key)
df = df.sort("account_id", "click_timestamp")
df.toPandas()

,account_id,buy,user_total_buys,user_total_visits,item_total_buys,item_total_visits,hist_buys,hist_visits
0,4b042feb-95d3-4710-b677-24d366895c44,1,0,0,5,27,0,0
1,4b042feb-95d3-4710-b677-24d366895c44,0,1,1,7,194,0,0
2,4b042feb-95d3-4710-b677-24d366895c44,1,1,2,6,36,1,1
3,4b042feb-95d3-4710-b677-24d366895c44,1,2,3,7,50,2,2


In [67]:
df = df_session.groupBy("account_id").mean()
df = df.filter(col("avg(buy)") > 0.5).filter(col("avg(buy)") < 1)
df.toPandas()#.sample(10)

,account_id,avg(buy),avg(visit)
0,a529c9a2-4844-4cc5-a50b-8ce586efda22,0.666667,1.0
1,8e8e220f-5aae-4c47-995e-b058d707c6e7,0.666667,1.0
2,b761feee-c7f0-4d68-98ab-8757bdb499fa,0.750000,1.0
3,ba275897-0566-4540-a6f4-80033fb944ca,0.750000,1.0
4,4b042feb-95d3-4710-b677-24d366895c44,0.750000,1.0
5,b6536359-9bff-470f-b144-61dab353d346,0.923077,1.0


In [61]:
df_session = spark.read.parquet("../output/ifood/dataset_3/session_train_0.10_k=0_s=1000/")
df_session = df_session.withColumn("visit", lit(1))
df_session = df_session.select("session_id","account_id", "merchant_id", "buy", "visit", "click_timestamp")
df_session = df_session.sort("account_id", "click_timestamp", "buy")


#df_session = df_session.filter(df_session.account_id == "3a3f6970-7c96-448c-a648-2eb0038d2f71")
df_session.filter(df_session.buy==1).limit(100).toPandas().sample(10)

,session_id,account_id,merchant_id,buy,visit,click_timestamp
53,ca118358-7620-4199-a216-06ce242711c7,b182295b-90b2-4053-8192-39cda3cbafbc,51c8a013-c1f7-45a7-b324-a943e4cdce68,1,1,2019-06-01 00:29:53
51,df5df220-aca4-4dc5-9758-037fee60ab35,ac555aa3-3fb0-4cd5-924e-e00b208440e3,92c5c3a6-40be-4a81-8969-3505b7bf16cc,1,1,2019-06-01 01:21:48
65,140dbcd2-578f-4ebd-bd7f-37bcd88576e3,b6536359-9bff-470f-b144-61dab353d346,92c5c3a6-40be-4a81-8969-3505b7bf16cc,1,1,2019-06-01 01:05:44
19,2d8364b8-7e67-43f8-8b39-5542c9d1d7c4,4b042feb-95d3-4710-b677-24d366895c44,16fb5cd2-a710-4fdb-9d74-f19da0734d3f,1,1,2019-06-01 00:49:08
84,bb304179-9a47-4355-98ba-9cb48aa75c98,d0e02cdc-9735-421e-aae9-0d96b24d7b9b,3a15e291-6c22-4d1e-9ce5-dbfb627da3a8,1,1,2019-06-01 00:49:42
28,3ce5d4de-8d64-4aba-8da5-5ebeec057b94,75619911-5279-4945-9d3d-a881bde3b4d8,3a15e291-6c22-4d1e-9ce5-dbfb627da3a8,1,1,2019-06-01 00:45:07
99,0d7896c5-dc64-4b42-87aa-c97709a4a25c,ec4f0cfd-b7a2-43e5-b17d-0f2802acf026,51c8a013-c1f7-45a7-b324-a943e4cdce68,1,1,2019-06-01 00:34:17
56,1008404b-088b-4cc2-a067-be5714b9637f,b624ab75-67ba-4e41-93cf-9cb4c93efea9,76c7216d-3fd4-452f-9bf2-b97fcf9f0c6c,1,1,2019-06-01 00:34:43
23,a97786dc-6539-4e55-a7cd-6cdf45b22910,4b8440a8-37e1-48df-afc1-c98acc23f796,92c5c3a6-40be-4a81-8969-3505b7bf16cc,1,1,2019-06-01 01:57:01
36,f1590a87-15d8-4b60-b531-d5c331317b8c,8455fafa-3a5d-4fbc-92d9-2f9e003b1ba5,16fb5cd2-a710-4fdb-9d74-f19da0734d3f,1,1,2019-06-01 01:32:43


In [91]:
win_user_total = Window.partitionBy('account_id').orderBy('click_timestamp')\
                    .rangeBetween(Window.unboundedPreceding, 0)

win_item_total = Window.partitionBy('merchant_id').orderBy('click_timestamp')

win_total = Window.partitionBy('merchant_id', 'account_id').orderBy('click_timestamp')\
                    .rangeBetween(Window.unboundedPreceding, 0)

#win_user_total_buys = Window.partitionBy('account_id').orderBy('click_timestamp')
#win_user_total_buys = Window.partitionBy('account_id').orderBy('click_timestamp')

df_session2 = df_session.withColumn("user_total_buys", F.sum(col("buy")).over(win_user_total)-col("buy"))
df_session2 = df_session2.withColumn("user_total_visits", F.sum(col("visit")).over(win_user_total)-col("visit"))
df_session2 = df_session2.withColumn("item_total_buys", F.sum(col("buy")).over(win_item_total)-col("buy"))
df_session2 = df_session2.withColumn("item_total_visits", F.sum(col("visit")).over(win_item_total)-col("visit"))

df_session2 = df_session2.withColumn("hist_buys", F.sum(col("buy")).over(win_total)-col("buy"))
df_session2 = df_session2.withColumn("hist_visits", F.sum(col("visit")).over(win_total)-col("visit"))

df_session2.filter(df_session.account_id == key).toPandas()#.head()
# df = df.withColumn("diff_prev_click", F.when(F.isnull(df.click_timestamp - df.prev_click_timestamp), 0)
#                               .otherwise(df.click_timestamp - df.prev_click_timestamp))
#df = df.withColumn("diff_clicked", ((df.click_timestamp.cast("long") - df.prev_click_timestamp.cast("long"))))
#df = df.withColumn("diff_clicked", (df.diff_clicked/60).cast("long"))
#df.select("session_id", "merchant_id", "prev_click_timestamp", "click_timestamp", "diff_clicked").show()

,session_id,account_id,merchant_id,buy,visit,click_timestamp,user_total_buys,user_total_visits,item_total_buys,item_total_visits,hist_buys,hist_visits
0,2d8364b8-7e67-43f8-8b39-5542c9d1d7c4,4b042feb-95d3-4710-b677-24d366895c44,16fb5cd2-a710-4fdb-9d74-f19da0734d3f,1,1,2019-06-01 00:42:49,0,0,5,27,0,0
1,2d8364b8-7e67-43f8-8b39-5542c9d1d7c4,4b042feb-95d3-4710-b677-24d366895c44,16fb5cd2-a710-4fdb-9d74-f19da0734d3f,1,1,2019-06-01 00:49:08,1,2,6,36,1,1
2,2d8364b8-7e67-43f8-8b39-5542c9d1d7c4,4b042feb-95d3-4710-b677-24d366895c44,16fb5cd2-a710-4fdb-9d74-f19da0734d3f,1,1,2019-06-01 01:11:13,2,3,7,50,2,2
3,2d8364b8-7e67-43f8-8b39-5542c9d1d7c4,4b042feb-95d3-4710-b677-24d366895c44,92c5c3a6-40be-4a81-8969-3505b7bf16cc,0,1,2019-06-01 00:48:21,1,1,7,194,0,0
